# Identifica produtos contidos em cada venda

Leitura do DataFrame de vendas e de custos

In [1]:
import pandas as pd

In [2]:
vendas_df = pd.read_csv('../data/gold/2025_11_final.csv')

In [5]:
custos_df = pd.read_excel('../data/bronze/custos_akira.xlsx')